In [1]:
from typing import List
from itertools import combinations

import asyncio
import aiohttp

import json
import time
import requests
import pandas as pd

import numpy as np 

In [2]:
BASE_OPENALEX = "https://api.openalex.org"

In [3]:
queries = [
    "dna kinetics",
    "graph theory",
    "dynamical systems",
    "monte carlo simulations",
    "origins of life",
    "stochastic processes",
    "polymer physics"
]

In [4]:
# Async version
async def fetch_papers_async(query: str, n_results=1000):
    query = "%20".join(query.split(" "))
    async with aiohttp.ClientSession() as session:
        tasks = []
        per_page = 200
        pages = (n_results // per_page) + 1
        for page in range(1, pages + 1):
            url = f"{BASE_OPENALEX}/works?search={query}&per-page={per_page}&page={page}"
            tasks.append(session.get(url))
        responses = await asyncio.gather(*tasks)
        results = []
        for response in responses:
            data = await response.json()
            results.extend(data['results'])
    return pd.DataFrame(results)

def search_papers(query: str, n_results=400) -> pd.DataFrame:
    """ Retrieve n_results out of a query to openalex """
    per_page = min(200, n_results)
    query = "%20".join(query.split(" "))
    url = f"{BASE_OPENALEX}/works?search={query}"
    params = {
        'per-page': per_page,
    }
    dfs = []
    for page in range(1, (n_results // per_page)+1):
        params['page'] = page
        response = requests.get(url, params=params).json()
        df = pd.json_normalize(response["results"])
        df = df.drop(columns=[col for col in df.columns if 'abstract' in col.lower()])
        dfs.append(df)            
    return pd.concat(dfs, ignore_index=True)

async def multi_search(queries: List[str], n_results=400) -> pd.DataFrame:
    """ Returns a dataframe with all retrieved papers for all queries """
    results = {}
    for query in queries: 
        results[query] = await fetch_papers_async(query, n_results=n_results)
    return pd.concat(list(results.values()), ignore_index=True)

def get_topics_set(results: pd.DataFrame):
    topics = results["topics"]
    topic_ids = []
    for topic in topics:
        for t in topic: 
            topic_ids.append(t["id"])
    return set(topic_ids)


In [5]:
results = await multi_search(queries, n_results=1000)

In [6]:
topics = get_topics_set(results)
t_idx = {i:t for i,t in enumerate(topics)}
idx_t = {t:i for i,t in t_idx.items()}

In [7]:
# create a matrix to index through topics 
mutualmatrix = np.zeros([len(topics), len(topics)])

In [8]:
for i, res in results.iterrows():
    # get p(x)
    for t in res["topics"]:
        id = idx_t[t["id"]]
        mutualmatrix[id, id] = mutualmatrix[id, id] + 1
    # get p(x, y)
    for ti, tj in combinations(res["topics"], r=2):
        idi, idj = idx_t[ti["id"]], idx_t[tj["id"]]
        mutualmatrix[idi, idj] = mutualmatrix[idi, idj] + 1
        mutualmatrix[idj, idi] = mutualmatrix[idj, idi] + 1


In [9]:
# Count non-zero elements and total sum in the mutual matrix
nonzero_count = (mutualmatrix != 0).sum().sum()
total_sum = mutualmatrix.sum().sum()
print(f"Number of non-zero elements in mutual matrix: {nonzero_count}")
print(f"Total sum of values in mutual matrix: {total_sum}")

Number of non-zero elements in mutual matrix: 18001
Total sum of values in mutual matrix: 63228.0


In [10]:
N = len(mutualmatrix)
probmatrix = mutualmatrix/N

In [11]:
pmimatrix = np.zeros_like(probmatrix)
for i,j in combinations(range(probmatrix.shape[0]), r=2):
    if probmatrix[i,j] > 0:
        pmimatrix[i,j] = np.log2(probmatrix[i,j]/(probmatrix[i,i]*probmatrix[j,j]))/(-np.log2(probmatrix[i,j]))
        pmimatrix[j,i] = pmimatrix[i,j]

In [12]:
results["score"] = 0
for i, work in results.iterrows():
    topics = work["topics"]
    for ti, tj in combinations(topics, r=2):
        # Fix: use ti and tj for the two different indices
        results.loc[i, "score"] += pmimatrix[idx_t[ti["id"]], idx_t[tj["id"]]]

/tmp/ipykernel_1437702/3381889387.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.28445786476728635' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[i, "score"] += pmimatrix[idx_t[ti["id"]], idx_t[tj["id"]]]


In [13]:
results = results.sort_values(by="score", ascending=True)